The purpose of this study is to create a detector of biased (non-random) ship placement which can be used to adjust shooting player strategy.

Until this point, all scripts have been assuming ideally random placement of ships on the board. However, in realworld scenarios, each player will have his own playstyle or subconscious preferences on how he distributes the ships. If the shooting player is able to decrypt this bias, or at least slightly adjust the probabilities of scoring hits in certain fields based on history of games played against the same adversary, he should be able to decrease the average amount of moves needed to win the game.

Because of the vast amount of possible game setups in standard, 10x10 board filled with 7 ships, for this study I decided to go easier route and reduced it to 6x6 board with 1x seg3 ship and 2x seg2 ships. The scripts which have prevously hardcoded values as for 10x10 board, were now made parametric. Hopedully, the solutions that I develop and test here with this easier setup, could be upscaled in future. 

In bias detector I will be creating a bunch of metrics describing the board setup (distance of ships to egdes, their clustering etc.)
These metrics will be calculated for each board setup in a set of randomly generated boards. Mean values and confidence intervals will be determined.
Then, same will be applied for another set of board setups, generated with some bias. For each metric, the detector will check if confidence interval determined from random boards overlaps the one determined from possibly biased boards. This way it can find statisticallty relevant bias.


First stage of this study will focus only on creating and processing the metrics, then using it to detect and quantify the differences between 2 sets of boards generated by chatGPT - one fully random, the other biased in an unknown way.

In [ ]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import copy

1. Ship orientation data:

    avg_2seg_orient - Average orientation of seg2 ships (+1 for vertical, -1 for horizontal, in random boards should be near 0)

    avg_3seg_orient - Average orientation of seg3 ships (+1 for vertical, -1 for horizontal, in random boards should be near 0)

    seg2_same_orient - True if both seg2 ships have same orientation
    
    seg2_3_same_orient - True if all ships (both seg2 + seg3) have the same orientation

In [ ]:

def get_avg_orients (groups):
    avg_2seg_orient = 0
    avg_3seg_orient = 0
    for ship in groups:
        if len(ship) == 2:
            (x1,y1), (x2,y2) = ship
            if x1 == x2:
                avg_2seg_orient += 1    # positive = vertical
            elif y1 == y2:
                avg_2seg_orient -= 1    # negative = horizontal
            else:
                raise ValueError("Invalid ship coordinates")
        if len(ship) == 3:
            (x1,y1), (x2,y2), (x3,y3) = ship
            if x1 == x2:
                avg_3seg_orient += 1
            elif y1 == y2:
                avg_3seg_orient -= 1
            else:
                raise ValueError("Invalid ship coordinates")
            
    if abs(avg_2seg_orient) == 2:
        seg2_same_orient = True
    else:
        seg2_same_orient = False

    if avg_2seg_orient * avg_3seg_orient > 0:
        seg2_3_same_orient = True
    else:
        seg2_3_same_orient = False

    return avg_2seg_orient, avg_3seg_orient, seg2_same_orient, seg2_3_same_orient



2. Distance to edge data:

    seg2_dist_to_left - Average distance from seg2 ship to the left edge of the board

    seg2_dist_to_right/upper/bottom - By analogy

    seg3_dist_to_... - by analogy

In [ ]:

def get_avg_distance_to_edges (groups, size):
    
    seg2_dist_to_left = 0
    seg2_dist_to_right = 0
    seg2_dist_to_upper = 0
    seg2_dist_to_bottom = 0

    seg3_dist_to_left = 0
    seg3_dist_to_right = 0
    seg3_dist_to_upper = 0
    seg3_dist_to_bottom = 0

    for ship in groups:
        if len(ship) == 2:
            (x1,y1), (x2,y2) = ship

            seg2_dist_to_left += min(y1, y2)
            seg2_dist_to_right += min(size - 1 - y1, size - 1 - y2)

            seg2_dist_to_upper += min(x1, x2)
            seg2_dist_to_bottom += min(size - 1 - x1, size - 1 - x2)
            
        if len(ship) == 3:
            (x1,y1), (x2,y2), (x3,y3) = ship

            seg3_dist_to_left += min(y1, y2, y3)
            seg3_dist_to_right += min(size - 1 - y1, size - 1 - y2, size - 1 - y3)

            seg3_dist_to_upper += min(x1, x2, x3)
            seg3_dist_to_bottom += min(size - 1 - x1, size - 1 - x2, size - 1 - x3)
    
    return (seg2_dist_to_left / 2, seg2_dist_to_right / 2, seg2_dist_to_upper / 2, seg2_dist_to_bottom / 2,
            seg3_dist_to_left, seg3_dist_to_right, seg3_dist_to_upper, seg3_dist_to_bottom)



In [ ]:

def get_avg_spreads (groups):

    def get_centroid (ship):
        xs = [x for x,y in ship]
        ys = [y for x,y in ship]
        return (sum(xs)/len(xs), sum(ys)/len(ys))

    def distance (p1, p2):
        return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5
    
    spread_2_2 = 0
    spread_2_3 = 0
    spread_all = 0

    #find pair of centroids of seg2 ships
    seg2_centroids = [get_centroid(ship) for ship in groups if len(ship) == 2]
    seg3_centroid = [get_centroid(ship) for ship in groups if len(ship) == 3]

    spread_2_2 += distance(seg2_centroids[0], seg2_centroids[1])
    spread_2_3 += distance(seg2_centroids[0], seg3_centroid[0])
    spread_2_3 += distance(seg2_centroids[1], seg3_centroid[0])
    spread_all = spread_2_2 + spread_2_3

    avg_spread_2_2 = spread_2_2
    avg_spread_2_3 = spread_2_3 / 2
    avg_spread_all = spread_all / 3

    return avg_spread_2_2, avg_spread_2_3, avg_spread_all


def get_ships_at_edges (groups, size):
    segments_of_ships_at_edges = 0

    for ship in groups:
        (x1, y1) = ship[0]
        (x2, y2) = ship[1]
            
        if x1 == x2:
            if x1 == 0 or x1 == size - 1:
                segments_of_ships_at_edges += len(ship)
        elif y1 == y2:
            if y1 == 0 or y1 == size - 1:
                segments_of_ships_at_edges += len(ship)
    
    return segments_of_ships_at_edges


def calculate_bias_metrics(known_board):
    binary_mask = np.logical_or(known_board == 3, known_board == 2).astype(int)

    groups = fun.group_adjacent_symbols(binary_mask, 1)

    avg_2seg_orient, avg_3seg_orient, seg2_same_orient, seg2_3_same_orient = get_avg_orients (groups)

    (seg2_dist_to_left, seg2_dist_to_right, seg2_dist_to_upper, seg2_dist_to_bottom, seg3_dist_to_left, seg3_dist_to_right, seg3_dist_to_upper, seg3_dist_to_bottom) = get_avg_distance_to_edges (groups, known_board.shape[0])

    avg_spread_2_2, avg_spread_2_3, avg_spread_all = get_avg_spreads (groups)

    segments_of_ships_at_edges = get_ships_at_edges (groups, known_board.shape[0])

    return {
        "avg_2seg_orient": avg_2seg_orient,
        "avg_3seg_orient": avg_3seg_orient,
        "seg2_same_orient": seg2_same_orient,
        "seg2_3_same_orient": seg2_3_same_orient,
        "seg2_dist_to_left": seg2_dist_to_left,
        "seg2_dist_to_right": seg2_dist_to_right,
        "seg2_dist_to_upper": seg2_dist_to_upper,
        "seg2_dist_to_bottom": seg2_dist_to_bottom,
        "seg3_dist_to_left": seg3_dist_to_left,
        "seg3_dist_to_right": seg3_dist_to_right,
        "seg3_dist_to_upper": seg3_dist_to_upper,
        "seg3_dist_to_bottom": seg3_dist_to_bottom,
        "avg_spread_2_2": avg_spread_2_2,
        "avg_spread_2_3": avg_spread_2_3,
        "avg_spread_all": avg_spread_all,
        "segments_of_ships_at_edges": segments_of_ships_at_edges
    }


def generate_board_from_empty(dim):
    board_hyp_lvl_7 = None
    while board_hyp_lvl_7 is None:
        known_board = np.array([[0] * dim for i in range(dim)])

        free_fields, hit_unsunk = fun.find_free_and_hit_fields(known_board)

        # Find different groups of adjacent '1' fields
        # (rare but possible scenario when we have multiple, non-adjacent ships hit but not sunk)
        groups = fun.group_adjacent_symbols(known_board, 1)

        minimum_lengths = fun.find_minimum_lengths(hit_unsunk, groups)

        # Counter of how many ships of each type are already sunk
        seg5_done = fun.n_segments(known_board, 5) / 5
        seg4_done = fun.n_segments(known_board, 4) / 4
        seg3_done = fun.n_segments(known_board, 3) / 3
        seg2_done = fun.n_segments(known_board, 2) / 2

        segments_done = {
            5: seg5_done,
            4: seg4_done,
            3: seg3_done,
            2: seg2_done
        }


        # 7 levels of hypothetical board, starting from placement of a 5-seg ship, ending with 2-segs
        n_seg_for_lvl = {
            1: 5,
            2: 4,
            3: 3,
            4: 3,
            5: 2,
            6: 2,
            7: 2
        }

        # Configs_levels = all valid combinations of a free field and orientation (hor/ver) for each ship length.
        configs_levels = random_board_generator.create_configs (known_board, free_fields, n_seg_for_lvl)

        board_hyp_lvl_7 = random_board_generator.generate_random_board(known_board, configs_levels, n_seg_for_lvl,
                                                                                seg5_done, seg4_done, seg3_done, seg2_done,
                                                                                hit_unsunk, minimum_lengths, game_rules_n_ships={5:0,4:0,3:1,2:2})
    return board_hyp_lvl_7


def calculate_confidence_interval(data, confidence=0.95):
    
    n = len(data)
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard error of the mean
    h = sem * stats.t.ppf((1 + confidence) / 2., n-1)  # Margin of error

    return (mean - h, mean + h)


def confidence_intervals_overlap(cA, cB):
    # cA and cB are tuples (lower_bound, upper_bound)
    return not (cA[1] < cB[0] or cB[1] < cA[0])